# FAISS 实现 rag 过滤

这个过滤，专业术语叫召回。即仅把相关文本喂给大模型。

FAISS： Facebook AI Similarity Search，是由脸书团队开源的针对聚类和相似性搜索库。

安装faiss:

```bash
conda install -c conda-forge faiss-gpu=1.7.3
```

cpu 版本：

```bash
pip install faiss-cpu
```

In [3]:
import shutup
shutup.please()

from langchain.vectorstores.faiss import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from dotenv import load_dotenv

load_dotenv()

corpus = [
    '武汉市，简称“汉”，别称江城，湖北省辖地级市、省会，副省级市、国家中心城市、超大城市，国务院批复确定的中国中部地区的中心城市。',  
    '武汉市下辖13个区，总面积8569.15平方千米。截至2022年末，常住人口1373.90万人，地区生产总值18866.43亿元。',
    '武汉市地处江汉平原东部、长江中游，长江及其最大支流汉水在此交汇，形成武汉三镇（武昌、汉口、汉阳）隔江鼎立的格局。',
    '湖北省，简称“鄂”，别名楚、荆楚，中华人民共和国省级行政区，省会武汉。',
    '湖北省辖12个地级市、1个自治州，39个市辖区、26个县级市、37个县（其中2个自治县）、1个林区。',
    '截至2022年末，湖北省常住人口5844万人，地区生产总值为53734.92亿元，人均地区生产总值为92059元。'
]

documents = [Document(page_content=cp) for cp in corpus]

embedding = OpenAIEmbeddings(model='text-embedding-ada-002')   # 这里使用本地模型会报错，参考  https://api.wlai.vip/ 获得代理api-key
documents_db = FAISS.from_documents(documents, embedding)
documents_db

可以看到现在向量数据库是一个object

In [5]:
question = '湖北省常驻人口是多少？'
retrieval_documents = documents_db.similarity_search(question, k=3) # 召回和问题最相似的3条文本
retrieval_documents

[Document(page_content='截至2022年末，湖北省常住人口5844万人，地区生产总值为53734.92亿元，人均地区生产总值为92059元。'),
 Document(page_content='武汉市下辖13个区，总面积8569.15平方千米。截至2022年末，常住人口1373.90万人，地区生产总值18866.43亿元。'),
 Document(page_content='湖北省辖12个地级市、1个自治州，39个市辖区、26个县级市、37个县（其中2个自治县）、1个林区。')]

In [6]:
question = '武汉三镇是哪三个？'
retrieval_documents = documents_db.similarity_search(question, k=3)
retrieval_documents

[Document(page_content='武汉市地处江汉平原东部、长江中游，长江及其最大支流汉水在此交汇，形成武汉三镇（武昌、汉口、汉阳）隔江鼎立的格局。'),
 Document(page_content='武汉市，简称“汉”，别称江城，湖北省辖地级市、省会，副省级市、国家中心城市、超大城市，国务院批复确定的中国中部地区的中心城市。'),
 Document(page_content='武汉市下辖13个区，总面积8569.15平方千米。截至2022年末，常住人口1373.90万人，地区生产总值18866.43亿元。')]

In [7]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model='qwen2:7b', temperature=0)

from langchain.chains.question_answering import load_qa_chain
chain = load_qa_chain(llm=llm, chain_type='stuff', verbose=True)  # 召回后文本就比较少了，可以全传给大模型

result = chain.run(input_documents=retrieval_documents, question=question)
result



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
武汉市地处江汉平原东部、长江中游，长江及其最大支流汉水在此交汇，形成武汉三镇（武昌、汉口、汉阳）隔江鼎立的格局。

武汉市，简称“汉”，别称江城，湖北省辖地级市、省会，副省级市、国家中心城市、超大城市，国务院批复确定的中国中部地区的中心城市。

武汉市下辖13个区，总面积8569.15平方千米。截至2022年末，常住人口1373.90万人，地区生产总值18866.43亿元。
Human: 武汉三镇是哪三个？

> Finished chain.

> Finished chain.


'武汉三镇分别是武昌、汉口和汉阳。'

其他的召回方法还有：

```python
# 相似性搜索，只返回最相关的k条数据
db.similarity_search(query, k=3)

# 相似性搜索，并返回相似度分数，分数越大越相关
db.similarity_search_with_score(query, k=3)

# 相似性搜索，分数越小越相关
db.similarity_search_with_relevance_scores(query)

# 最大边际搜索，优化了与查询的相似性，和文档之间的多样性。
db.max_marginal_relevance_search(query, k=3)
```

In [8]:
question = '武汉三镇是哪三个？'
retrieval_documents = documents_db.similarity_search_with_score(question, k=3)
retrieval_documents

[(Document(page_content='武汉市地处江汉平原东部、长江中游，长江及其最大支流汉水在此交汇，形成武汉三镇（武昌、汉口、汉阳）隔江鼎立的格局。'),
  0.8680007),
 (Document(page_content='武汉市，简称“汉”，别称江城，湖北省辖地级市、省会，副省级市、国家中心城市、超大城市，国务院批复确定的中国中部地区的中心城市。'),
  0.85329753),
 (Document(page_content='武汉市下辖13个区，总面积8569.15平方千米。截至2022年末，常住人口1373.90万人，地区生产总值18866.43亿元。'),
  0.84345716)]

有了分数之后，就可以再在代码里附带判断，比如大于或者小于多少分的召回结果我们就不要了。

In [9]:
question = '武汉三镇是哪三个？'
retrieval_documents = documents_db.similarity_search_with_relevance_scores(question, k=3)
retrieval_documents

[(Document(page_content='武汉市地处江汉平原东部、长江中游，长江及其最大支流汉水在此交汇，形成武汉三镇（武昌、汉口、汉阳）隔江鼎立的格局。'),
  0.3862308287355567),
 (Document(page_content='武汉市，简称“汉”，别称江城，湖北省辖地级市、省会，副省级市、国家中心城市、超大城市，国务院批复确定的中国中部地区的中心城市。'),
  0.39662752903252707),
 (Document(page_content='武汉市下辖13个区，总面积8569.15平方千米。截至2022年末，常住人口1373.90万人，地区生产总值18866.43亿元。'),
  0.40358572084058464)]

In [10]:
question = '武汉三镇是哪三个？'
retrieval_documents = documents_db.max_marginal_relevance_search(question, k=3)
retrieval_documents

[Document(page_content='武汉市地处江汉平原东部、长江中游，长江及其最大支流汉水在此交汇，形成武汉三镇（武昌、汉口、汉阳）隔江鼎立的格局。'),
 Document(page_content='湖北省辖12个地级市、1个自治州，39个市辖区、26个县级市、37个县（其中2个自治县）、1个林区。'),
 Document(page_content='武汉市下辖13个区，总面积8569.15平方千米。截至2022年末，常住人口1373.90万人，地区生产总值18866.43亿元。')]